# BeeCryption
First, notice that every operation is linear in the entries of the flowerfields: there are only permutations and xors.
This means we'll be able to run the encryption symbolically with polynomials over GF(256), and compare them to the actual encryption (because we know part of the plaintext).

In [1]:
class BeeHiveSymbolic:
    """ Surrounding their beehive, the bees can feast on six flower fields with six flowers each. """
    def __init__(self, key):
        """ Initialise the bee colony. """
        self.fields = self.plant_flowers(key)
        self.nectar = None
        self.collect()
        
    def plant_flowers(self, key):
        """ Plant flowers around the beehive. """
        try:
            if type(key) == str:
                key = bytes.fromhex(key)
            return [FlowerField(key[i:i+6]) for i in range(0,36,6)]
        except:
            raise ValueError('Invalid Key!')

    def collect(self):
        """ Collect nectar from the closest flowers. """
        A,B,C,D,E,F = [i.flowers for i in self.fields]
        self.nectar = [A[2] + B[4], B[3] + C[5], C[4] + D[0], D[5] + E[1], E[0] + F[2], F[1] + A[3]]

    def cross_breed(self):
        """ Cross-breed the outermost bordering flowers. """
        def swap_petals(F1: FlowerField, F2: FlowerField, i1, i2):
            """ Swap the petals of two flowers. """
            F1.flowers[i1], F2.flowers[i2] = F2.flowers[i2], F1.flowers[i1]

        A, B, C, D, E, F = self.fields
        swap_petals(A,B,1,5)
        swap_petals(B,C,2,0)
        swap_petals(C,D,3,1)
        swap_petals(D,E,4,2)
        swap_petals(E,F,5,3)
        swap_petals(F,A,0,4)
        
    def pollinate(self):
        """ Have the bees pollinate their flower fields (in order). """
        bees = self.nectar[:]

        for i in range(6):
            bees = [self.fields[i].flowers[k] + bees[k] for k in range(6)]
            self.fields[i].flowers = bees

    def stream(self, n=1):
        """ Produce the honey... I mean keystream! """
        buf = []
        # Go through n rounds
        for _ in range(n):
            # Go through 6 sub-rounds
            for field in self.fields:
                field.rotate()
                self.cross_breed()
                self.collect()
                self.pollinate()
            # Collect nectar
            self.collect()
            buf += self.nectar
        return buf

    def encrypt(self, msg):
        """ Beecrypt your message! """
        beeline = self.stream(n = -(-len(msg)//6))
        return [beeline[i] + msg[i] for i in range(len(msg))]

class FlowerField:
    """ A nice field perfectly suited for a total of six flowers. """
    def __init__(self, flowers):
        """ Initialise the flower field. """
        self.flowers = [i for i in flowers]

    def rotate(self):
        """ Crop-rotation is important! """
        self.flowers = [self.flowers[-1]] + self.flowers[:-1]
        
    def __repr__(self):
        return 'F' + repr(self.flowers)

Changing `^^` to `+` and removing a few checks to make sure it doesn't use `bytes.fromhex` on the polynomials is enough to make it able to run symbolically.

In [2]:
K.<x> = GF(256)

K is the field of order 256 generated by x; the elements look like x^5 + x^2 + 1, where the coefficient of x^n is the n-th bit of the integer represented by the polynomial. Addition in K is xor in the integers.

In [3]:
P = PolynomialRing(K, 36, names = ','.join(f'f{i}{j}' for i in range(6) for j in range(6)))
print(P.gens())

(f00, f01, f02, f03, f04, f05, f10, f11, f12, f13, f14, f15, f20, f21, f22, f23, f24, f25, f30, f31, f32, f33, f34, f35, f40, f41, f42, f43, f44, f45, f50, f51, f52, f53, f54, f55)


`fij` is the `j`-th coefficient of the `i`-th flowerfield.

In [4]:
B = BeeHiveSymbolic(P.gens())
B.fields

[F[f00, f01, f02, f03, f04, f05],
 F[f10, f11, f12, f13, f14, f15],
 F[f20, f21, f22, f23, f24, f25],
 F[f30, f31, f32, f33, f34, f35],
 F[f40, f41, f42, f43, f44, f45],
 F[f50, f51, f52, f53, f54, f55]]

In [5]:
msg = b'Bees make honey, but they also made the Bee Movie... f'
m = [*map(K.fetch_int, msg)] # K.fetch_int(0b01101) = x^3 + x^2 + 1

In [6]:
symc = B.encrypt(m)

In [7]:
enc = bytes.fromhex('b8961e85e54e357cf1bb18550e9d397cb6e522e386a837a970c71e02a353eddb9117713e60aa8f764e4525f86898e379fce195437ec59202a5b715334b3295b9f9c9280b2de048183f1a9581860b852167102c1c4ec2897b59e360b5ba37d90b60f23b2ad47c04782a6be3bf')
c = [*map(K.fetch_int, enc)]

We know that the `i`-th value of `symc` should be the same as the `i`-th value of `c`, that is `symc[i] - c[i] = 0`.
The actual values of the `kij` are the common roots of all these polynomials: we will use Gröbner basis to find them.

In [8]:
solve = [*Ideal([a - b for a, b in zip(symc, c)]).groebner_basis()]
solve[:3]

[f00 + (x^7 + x^6 + x^5 + x^4 + x^3 + x),
 f01 + (x),
 f02 + (x^6 + x^5 + x^3 + x + 1)]

In [9]:
key = [-s.monomial_coefficient(P(1)) for s in solve] # x + a = 0 <=> x = -a

In [10]:
B = BeeHiveSymbolic(key) # it also works on non symbolic entries, wonderful!
enc.startswith(bytes([z.integer_representation() for z in B.encrypt(m)]))

True

In [11]:
B = BeeHiveSymbolic(key)
bytes([z.integer_representation() for z in B.encrypt(c)]) # encryption and decryption are the same, because xor.

b'Bees make honey, but they also made the Bee Movie... flag{th3s3_mumbl3_b33s_4r3_h4rd_t0_und3rst4nd_y0u_kn0w}'